<a href="https://colab.research.google.com/github/willystw/fastai-learning/blob/fraud-detection/fraud-detection/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#hide
!pip install -Uqq fastbook waterfallcharts treeinterpreter dtreeviz
!pip install --upgrade --force-reinstall --no-deps kaggle
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 58 kB 4.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=332d564d787a15dddc3e4d43b328c027a243e5c7c2e84a5f38105e21618b18d6
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Mounted at /content/gdrive


Import kaggle.json before executing the next step

In [9]:
!chmod 600 kaggle.json
!mv kaggle.json ~/.kaggle/

In [10]:
#hide
from fastbook import *
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [27]:
path = URLs.path('fraud_detection')
path

Path('.')

In [25]:
#hide
Path.BASE_PATH = path

In [32]:
if not path.exists():
    path.mkdir(parents=true)
    api.dataset_download_cli(dataset='volodymyrgavrysh/fraud-detection-bank-dataset-20k-records-binary', path=path)
    file_extract(path/'fraud-detection-bank-dataset-20k-records-binary.zip')

path.ls()


fraud-detection-bank-dataset-20k-records-binary.zip: Skipping, found more recently modified local copy (use --force to force download)


NameError: ignored